In [1]:
import pandas as pd
import numpy as np


In [2]:
df1 = pd.read_csv('data.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)

df2 = pd.read_csv('data.csv',index_col=False, encoding='iso-8859-1', 
                                warn_bad_lines=True, error_bad_lines=False)


C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\moshi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df1.label[df1.label == 1]="acpt"
df1.label[df1.label == 0]="unac"

df2.label[df2.label == 1]="acpt"
df2.label[df2.label == 0]="unac"

C:\Users\moshi\AppData\Local\Temp/ipykernel_13300/3898086040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.label[df1.label == 1]="acpt"
C:\Users\moshi\AppData\Local\Temp/ipykernel_13300/3898086040.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.label[df2.label == 1]="acpt"


In [4]:
# taking acceptable and deleting unacceptable
df3 = df1[~df1.label.str.contains('acpt')]

In [5]:
# randomly picking 250 rows
df3 = df3.sample(n=250, replace=True)

In [6]:
df3["label"].value_counts()

unac    250
Name: label, dtype: int64

In [7]:
# taking unacceptable and deleting unacceptable
df4 = df2[~df2.label.str.contains("unac")]

In [8]:
# randomly picking 250 rows
df4 = df4.sample(n=250, replace=True)

In [9]:
df4["label"].value_counts()

acpt    250
Name: label, dtype: int64

In [10]:
#concating acceptable and unacceptable
frames = [df3, df4]

temp = pd.concat(frames)

In [11]:
temp["label"].value_counts()

unac    250
acpt    250
Name: label, dtype: int64

In [12]:
temp.head()

,label,text
2539,unac,I put the book from Edna to Sally.
744,unac,Frank promised the men all to leave.
1284,unac,Tony has a Fiat to yearn for a tall nurse.
4970,unac,What did John wonder who would win?
527,unac,The box contained the ball to the ground.


In [13]:
temp.tail()

,label,text
472,acpt,They tried to all like John.
1622,acpt,How surprisingly well he dances!
4666,acpt,She was sent to Seoul.
4371,acpt,"John may drink water, and Bill drink beer."
326,acpt,"Louise likes not being happy, doesn't she?"


In [14]:
temp["label"].value_counts()

unac    250
acpt    250
Name: label, dtype: int64

In [15]:
text = temp['text'].values.tolist()
emotion= temp['label'].tolist()

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text , emotion, test_size=.2, random_state=42)

In [17]:
import ktrain
from ktrain import text
MODEL_NAME = 'bert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500)


In [18]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 8
	95percentile : 15
	99percentile : 19


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 8
	95percentile : 14
	99percentile : 22


In [19]:
model = t.get_classifier()

In [20]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [21]:
learner.fit_onecycle(2e-5, 5)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/5
67/67 [==============================] - 1182s 18s/step - loss: 0.7065 - accuracy: 0.4575 - val_loss: 0.6940 - val_accuracy: 0.4700
Epoch 2/5
67/67 [==============================] - 1740s 26s/step - loss: 0.6740 - accuracy: 0.5875 - val_loss: 0.6552 - val_accuracy: 0.7100
Epoch 3/5
67/67 [==============================] - 1442s 21s/step - loss: 0.5283 - accuracy: 0.7550 - val_loss: 0.6754 - val_accuracy: 0.5800
Epoch 4/5
67/67 [==============================] - 1034s 15s/step - loss: 0.1363 - accuracy: 0.9725 - val_loss: 0.9668 - val_accuracy: 0.6400
Epoch 5/5
67/67 [==============================] - 1034s 15s/step - loss: 0.0569 - accuracy: 0.9875 - val_loss: 1.0863 - val_accuracy: 0.6300


In [22]:
class_names=["acpt","unac"]

In [23]:
learner.validate(val_data=(val), class_names=class_names)

              precision    recall  f1-score   support

        acpt       0.63      0.76      0.69        54
        unac       0.63      0.48      0.54        46

    accuracy                           0.63       100
   macro avg       0.63      0.62      0.62       100
weighted avg       0.63      0.63      0.62       100



array([[41, 13],
       [24, 22]], dtype=int64)